In [1]:

# coding: utf-8

# In[1]:

import pandas as pd
import datetime
import numpy as np
import os
import glob
import gc
import logging
logging.basicConfig(filename='/home/jian/BL_weekly_crontab/cron_6_weekly_audience_tracker/weekly_audience_tracker.log', level=logging.INFO)

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()
print("Worker Start: "+str(datetime.datetime.now()))


# In[2]:

last_sturday = (datetime.datetime.now()-datetime.timedelta(days=(datetime.datetime.now().weekday()+2))).date()
print("last_sturday: "+str(last_sturday))

last_day_of_2018Q4=datetime.date(2019,2,2)

year_of_quarter=(last_sturday-last_day_of_2018Q4).days/(52*7)
year_of_quarter=str(int(2019+np.floor(year_of_quarter)))
print("Year",year_of_quarter)


quarter_of_quarter=(last_sturday-last_day_of_2018Q4).days/7
quarter_of_quarter=np.floor(quarter_of_quarter/13)%4
quarter_of_quarter=str(int(1+quarter_of_quarter))
print("Quarter",quarter_of_quarter)

str_current_quarter=year_of_quarter+"_Q"+quarter_of_quarter

print(str_current_quarter)

current_week=int((last_sturday-last_day_of_2018Q4).days/7%13)
print("current_week",current_week)

if current_week==0:
    quarter_of_quarter=int(quarter_of_quarter)-1+4
    year_of_quarter=str(int(year_of_quarter)-1)
    str_current_quarter=year_of_quarter+"_Q"+str(quarter_of_quarter)
    current_week=13
    print("Quarter",quarter_of_quarter)
    print(str_current_quarter)
    print("current_week",current_week)
    
    
quarter_of_quarter


# In[3]:

current_quarter_beginning=last_day_of_2018Q4+datetime.timedelta(days=(int(year_of_quarter)-2019)*52*7+                                                                (int(quarter_of_quarter)-1)*13*7+1)
print("current_quarter_beginning: "+str(current_quarter_beginning))

current_quarter_beginning_last_year=current_quarter_beginning-datetime.timedelta(days=52*7)
print("current_quarter_beginning_last_year: "+str(current_quarter_beginning_last_year))


# In[4]:

all_weekly_new_sign_ups=list(recursive_file_gen("/home/jian/BigLots/"))
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if "aster" in x.lower()]
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if x[-4:]==".txt"]
all_weekly_new_sign_ups=[x for x in all_weekly_new_sign_ups if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
all_weekly_new_sign_ups.sort()
print(len(all_weekly_new_sign_ups))

Worker Start: 2020-02-05 14:18:26.138006
last_sturday: 2020-02-01
Year 2020
Quarter 1
2020_Q1
current_week 0
Quarter 4
2019_Q4
current_week 13
current_quarter_beginning: 2019-11-03
current_quarter_beginning_last_year: 2018-11-04
59


In [2]:
new_sign_ups_piece_1="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip"
new_sign_ups_piece_2="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip"
all_weekly_new_sign_ups.append(new_sign_ups_piece_1)
all_weekly_new_sign_ups.append(new_sign_ups_piece_2)
print(len(all_weekly_new_sign_ups))


# In[5]:

list_new_ids=[]
for file in all_weekly_new_sign_ups:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df=df[df['sign_up_date']>=str(current_quarter_beginning_last_year)]
    list_new_ids=list_new_ids+df['customer_id_hashed'].unique().tolist()
list_new_ids=list(set(list_new_ids))


# In[6]:

audience_files_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_"+str_current_quarter.replace("_","")+"/upload_files/"


61


In [3]:
audience_files_this_quarter=glob.glob(audience_files_folder+"*.csv")
len(audience_files_this_quarter)


audience_files_this_quarter_C=[x for x in audience_files_this_quarter if os.path.basename(x)[0]=="C"]
audience_files_this_quarter_T=[x for x in audience_files_this_quarter if os.path.basename(x)[0]=="T"]
print(len(audience_files_this_quarter_C),len(audience_files_this_quarter_T))

set_Control=set()
for file in audience_files_this_quarter_C:
    df=pd.read_csv(file,dtype=str,usecols=["customer_id_hashed"])
    set_Control=set_Control.union(set(df['customer_id_hashed'].tolist()))
print(len(set_Control))


df_audience=pd.DataFrame()

for file in audience_files_this_quarter_T:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df=df[~df['customer_id_hashed'].isin(set_Control)]
    df_audience=df_audience.append(df)
df_audience=df_audience.drop_duplicates()

print(df_audience.shape," | ",df_audience['customer_id_hashed'].nunique())

89 89
541818
(21435770, 2)  |  21435770


In [4]:
df_audience

,customer_id_hashed,segment
0,de4d254f85f84828db782a91857b389542ca253714330b...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
1,ea4e3e1699773386da4e152d8b9b93d10e21383ed231e0...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
2,8cc90ffdf1dd0b1eb6c8ec144a69bcee631d3d61a2983c...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
3,b085e358cb472d2b7a348043ff9b749d1265b8ce5fcb79...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
4,70bf02d737994c14f741d75b4a64c56b3839bbed27290e...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
...,...,...
1434539,640ca5854553852334535ee82506ff3c6066ffe4e68425...,T_Lapsed (19-48)_Density(M)_SOTF_AllExpCluster...
1434540,b2896b6e87daa54b67e4e9a61eda3771fb2ef1e274deac...,T_Lapsed (19-48)_Density(M)_SOTF_AllExpCluster...
1434541,336934ac057abc49fc789c42cd424c443c55e801a80957...,T_Lapsed (19-48)_Density(M)_SOTF_AllExpCluster...
1434542,e5b26a43831a9379610458f49cca2ddadd9281430facbe...,T_Lapsed (19-48)_Density(M)_SOTF_AllExpCluster...


In [5]:
df_clean_segment=df_audience[['segment']].drop_duplicates()
df_clean_segment_1_lapsed=df_clean_segment[df_clean_segment['segment'].str.contains("48")]
df_clean_segment_2_in_18=df_clean_segment[~df_clean_segment['segment'].str.contains("48")]

df_clean_segment_1_lapsed['segment_2']="Lapsed_19_48"
df_clean_segment_2_in_18['segment_2']=df_clean_segment_2_in_18['segment'].apply(lambda x: x.split("Score(")[1][0]+"_"+x.split("_2019Q4_RFM")[0][-1])
df_clean_segment=df_clean_segment_1_lapsed.append(df_clean_segment_2_in_18)
df_clean_segment.shape

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(89, 2)

In [6]:
print(df_audience.shape)
df_audience_total_count_original=df_audience.groupby("segment")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
df_audience=pd.merge(df_audience,df_clean_segment,on="segment",how="left")
print(df_audience.shape)
df_audience_total_count=df_audience.groupby("segment_2")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
del df_audience['segment']
df_audience=df_audience.rename(columns={"segment_2":"segment"})

print(df_audience_total_count.shape)
df_audience_total_count


# In[10]:

df_audience_total_count['ids_total'].sum()

(21435770, 2)
(21435770, 3)
(7, 2)


21435770

In [14]:

list_POS_this_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2016" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2017" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2018" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "histor" not in x.lower()]
list_POS_this_year_in_quarter.sort()
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning)]
# date only >=
print("len(list_POS_this_year_in_quarter): "+str(len(list_POS_this_year_in_quarter)))


# In[12]:

last_year_end_week_in_quarter=last_sturday-datetime.timedelta(days=52*7)

if year_of_quarter=="2019" and (quarter_of_quarter=="3" or quarter_of_quarter=="4"): 
    list_POS_last_year_in_quarter=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(current_quarter_beginning_last_year).replace("-","")]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(last_year_end_week_in_quarter).replace("-","")]
    print(1)
elif year_of_quarter=="2020" and quarter_of_quarter=="1": 
    list_POS_last_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "history" not in x.lower()]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2019" in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2019-02-09" not in x]

    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]<=str(last_year_end_week_in_quarter)]
    list_POS_last_year_in_quarter.append("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt")
    print(2)
    
else:
    list_POS_last_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2018" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2017" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "2016" not in x]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if "history" not in x.lower()]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning_last_year)]
    list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStorm_")[1][:10]<=str(last_year_end_week_in_quarter)]
    print(3)
print("len(list_POS_last_year_in_quarter):"+str(len(list_POS_last_year_in_quarter)))

len(list_POS_this_year_in_quarter): 13
3
len(list_POS_last_year_in_quarter):5


In [15]:
type(quarter_of_quarter)

int

In [8]:
# In[13]:

list_POS_this_year_in_quarter=list(recursive_file_gen("/home/jian/BigLots/"))
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "Daily" in x and x[-4:]==".txt"]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2016" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2017" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "2018" not in x]
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if "histor" not in x.lower()]
list_POS_this_year_in_quarter.sort()
list_POS_this_year_in_quarter=[x for x in list_POS_this_year_in_quarter if x.split("/MediaStorm_")[1][:10]>=str(current_quarter_beginning)]
# date only >=
print("len(list_POS_this_year_in_quarter): "+str(len(list_POS_this_year_in_quarter)))


# In[14]:

if len(list_POS_this_year_in_quarter)!=current_week or len(list_POS_last_year_in_quarter)!=current_week:
    print(str(datetime.datetime.now())+" | Error: POS file lenthg this year and last year different")
    print(str(datetime.datetime.now())+" | current_week "+str(current_week))
    
    print(str(datetime.datetime.now())+" | list_POS_this_year_in_quarter "+str(len(list_POS_this_year_in_quarter)))
    print(str(datetime.datetime.now())+" | list_POS_last_year_in_quarter "+str(len(list_POS_last_year_in_quarter)))
    # break


# In[15]:

print(datetime.datetime.now())

len(list_POS_this_year_in_quarter): 13
2020-02-05 14:41:46.508768 | Error: POS file lenthg this year and last year different
2020-02-05 14:41:46.509076 | current_week 13
2020-02-05 14:41:46.509356 | list_POS_this_year_in_quarter 13
2020-02-05 14:41:46.509628 | list_POS_last_year_in_quarter 5
2020-02-05 14:41:46.509988


In [9]:
list_POS_last_year_in_quarter

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-05/MediaStormDailySales20190108-121531-923.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-12/MediaStormDailySales20190115-113411-782.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-19/MediaStormDailySales20190122-113619-658.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-01-26/MediaStormDailySales20190129-111506-486.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-02/MediaStormDailySales20190205-111300-808.txt']

In [12]:
list_POS_last_year_in_quarter=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]>=str(current_quarter_beginning_last_year).replace("-","")]
list_POS_last_year_in_quarter=[x for x in list_POS_last_year_in_quarter if x.split("/MediaStormDailySalesHistory")[1][:8]<=str(last_year_end_week_in_quarter).replace("-","")]
len(list_POS_last_year_in_quarter)

13

In [10]:
str(current_quarter_beginning_last_year).replace("-","")

'20181104'

In [11]:
str(last_year_end_week_in_quarter).replace("-","")

'20190202'